In [1]:
import tensorflow as tf
from functools import partial
from libs.pipeline_utils import build_sampler_env, sample
from libs.pipeline_utils import wrap_list_with_score, simple_probability_pipeline, probability
from libs.utils import load_dictionary, load_transformer, top_best, load_dis

In [2]:
n_items = 1000
batch_size = 64
enc_seq_len = 64
dec_seq_len = 64

In [3]:
load_dir = "./models/shm_c1"
sampler_args = build_sampler_env(load_dir, batch_size, enc_seq_len, dec_seq_len)

INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519
INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519


In [4]:
dictionary = load_dictionary("./data/")
transformer = load_transformer(load_dir)

In [5]:
dis_be = load_dis('./models/', 'believability')
dis_st = load_dis('./models/', 'style')

Using TensorFlow backend.


In [6]:
be_f = partial(probability, model=dis_be, transformer=transformer)
st_f = partial(probability, model=dis_st, transformer=transformer)

In [28]:
sample_f = partial(sample, *sampler_args, 
                   dictionary=dictionary, 
                   transformer=transformer, 
                   n_items=n_items,
                   batch_size=64)

In [30]:
seed_phrase = 'Прогуляемся по крышам'
simple_probability_pipeline(seed_phrase, sample_f, [be_f, st_f], topn=0.02)

[('вс это я не мог уступить его.', 0.055512903658601509),
 ('стало быть, вс равно.', 0.015953690253754149),
 ('вс это мирно вздохнуло.', 0.010838032032603528),
 ('помешанная дуэль.', 0.0054163668836402223),
 ('даже както радуется играю...', 0.0050736914676916822),
 ('вообще, господин кармазинов.', 0.0042690126009892193),
 ('вс равно.', 0.0040625381958947493),
 ('этот фазис есть первый бежать.', 0.0039521296100136494),
 ('поднялись еще одиннадцати.', 0.0035245433235120693),
 ('тотчас же и гадко.', 0.0034199389962527443),
 ('долго еще обнимала руки.', 0.0034150487911929761),
 ('его вс поняла?', 0.0033131582661558856),
 ('опятьтаки крокодиль!', 0.0032488685381335536),
 ('оставим его, хотите выпить.', 0.0026585096284126308),
 ('весь оба пропавший прокурор.', 0.0023842989372000745),
 ('он стоял перед обеими руками.', 0.0023347105896932008),
 ('сам слушает меня.', 0.0022978681230691744),
 ('старуха в исступлении сила  гм.', 0.0022110957676904491),
 ('как бы вы это вселили?', 0.00216907456127